<a href="https://colab.research.google.com/github/matabesu/openai/blob/main/Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
# Install packages
!pip install git+https://github.com/openai/whisper.git


In [ ]:
import os

# Add folders
checkContentFolder = os.path.exists("content")
checkDownLoadFolder = os.path.exists("download")
if not checkContentFolder:
  os.mkdir("content")
if not checkDownLoadFolder:
  os.mkdir("download")
  

In [ ]:
import whisper

fileName = "question.mp3"
lang = "ja"
model = whisper.load_model("base")

# Load audio
audio = whisper.load_audio(f"content/{fileName}")
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Output the recognized text
options = whisper.DecodingOptions(language=lang, without_timestamps=True)
result = whisper.decode(model, mel, options)
print(result.text)

# Write into a text file
with open(f"download/{fileName}.txt", "w") as f:
  f.write(f"▼ Transcription of {fileName}\n")
  f.write(result.text)


In [ ]:
import whisper

fileName = "muddy_test.wav"
lang = "ja"
model = whisper.load_model("base")

# Load audio
audio = whisper.load_audio(f"content/{fileName}")
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Output the recognized text
options = whisper.DecodingOptions(language=lang, without_timestamps=True)
result = whisper.decode(model, mel, options)
print(result.text)

# Write into a text file
with open(f"download/{fileName}.txt", "w") as f:
  f.write(f"▼ Transcription of {fileName}\n")
  f.write(result.text)
  

In [ ]:
from google.colab import files
!zip -r download.zip download
files.download("download.zip")

updating: download/ (stored 0%)
updating: download/question.mp3.txt (deflated 36%)
  adding: download/muddy_test.wav.txt (deflated 41%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import whisper
model = whisper.load_model("base")

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
from IPython.display import Audio

Audio("content/question.mp3")

In [ ]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("content/question.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

In [ ]:
! pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
model = whisper.load_model("base")

In [ ]:
model.device

In [ ]:
import os

# Add folders
checkContentFolder = os.path.exists("content")
checkDownLoadFolder = os.path.exists("download")
if not checkContentFolder:
  os.mkdir("content")
if not checkDownLoadFolder:
  os.mkdir("download")

In [ ]:
# 長時間用音声解析
fileName = "test.mp3"

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(f"content/{fileName}")

outputTextsArr = []
while audio.size > 0:
  tirmedAudio = whisper.pad_or_trim(audio)
  # trimedArray.append(tirmedAudio)
  startIdx = tirmedAudio.size
  audio = audio[startIdx:]

  # make log-Mel spectrogram and move to the same device as the model
  mel = whisper.log_mel_spectrogram(tirmedAudio).to(model.device)

  # detect the spoken language
  _, probs = model.detect_language(mel)
  # print(f"Detected language: {max(probs, key=probs.get)}")

  # decode the audio
  options = whisper.DecodingOptions()
  result = whisper.decode(model, mel, options)

  # print the recognized text
  outputTextsArr.append(result.text)

outputTexts = ' '.join(outputTextsArr)
print(outputTexts)

# Write into a text file
with open(f"download/{fileName}.txt", "w") as f:
  f.write(f"▼ Transcription of {fileName}\n")
  f.write(outputTexts)

In [ ]:
from google.colab import files

!zip -r download.zip download
files.download("download.zip")

In [ ]:
pip install srt
pip install pysrt

In [ ]:
import torch
import whisper
import os  #ファイル操作用

from datetime import timedelta
from srt import Subtitle
import srt 
import pysrt

filepath = f"content/test.mp3"
lang = "ja" #音声ファイルの言語（ja=日本語）
basename = os.path.splitext(os.path.basename(filepath))[0] #音声ファイルの名前（拡張子なし）
model = whisper.load_model("small") #モデルサイズの指定(上の表参照）

#audioファイルを読み込む
audio = whisper.load_audio(file=filepath)

#音声認識
result = model.transcribe(audio, verbose=True, language=lang)

segments = result["segments"]

subs = []
for data in segments:
    index = data["id"] + 1
    start = data["start"]
    end = data["end"]
    text = data["text"]

    sub = Subtitle(
        index = 1,
        start = timedelta(seconds = timedelta(seconds=start).seconds, microseconds=timedelta(seconds=start).microseconds),
        end = timedelta(seconds = timedelta(seconds=end).seconds, microseconds=timedelta(seconds=end).microseconds), 
        content=text, 
        proprietary=''
        )
 
    subs.append(sub)

#格納した情報をSRTファイルとして書き出す
with open(f"{basename}.srt", mode="w", encoding="utf-8") as f:
    f.write(srt.compose(subs))

#SRTファイルから必要な情報だけ取り出してtxtファイルで保存する
subrip = pysrt.open(f"{basename}.srt")
f_out = open(f"{basename}.txt", mode="w", encoding="utf-8")

#テキスト（IDとタイムスタンプ無し）
for sub in subrip:
    f_out.write(sub.text + '\n')
    
#タイムスタンプ、テキスト（ID無し）
#for sub in subrip:
 #   f_out.write(str(sub.start) + ' --> ' + str(sub.end) + '\n')
 #   f_out.write(sub.text + '\n')